<a href="https://colab.research.google.com/github/Ankur7470/nlp-assignment2/blob/main/NLP_assgn2_task2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 65.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 17.2 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.14.1
    Uninstalling scipy-1.14.1:
      Successfully uninstalled scipy-1.14.1


## Importing the necessary libraries

In [3]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from gensim.models import Word2Vec
import torch
from torch.utils.data import Dataset, DataLoader

## EDA

### Loading the Sentiment Analysis Dataset

In [4]:
sentiment_data = pd.read_csv("sentimentdataset.csv")

### Information about the Dataset

In [5]:
sentiment_data.head()

,Unnamed: 0.1,Unnamed: 0,Text,Sentiment,Timestamp,User,Platform,Hashtags,Retweets,Likes,Country,Year,Month,Day,Hour
0,0,0,Enjoying a beautiful day at the park! ...,Positive,2023-01-15 12:30:00,User123,Twitter,#Nature #Park,15.0,30.0,USA,2023,1,15,12
1,1,1,Traffic was terrible this morning. ...,Negative,2023-01-15 08:45:00,CommuterX,Twitter,#Traffic #Morning,5.0,10.0,Canada,2023,1,15,8
2,2,2,Just finished an amazing workout! 💪 ...,Positive,2023-01-15 15:45:00,FitnessFan,Instagram,#Fitness #Workout,20.0,40.0,USA,2023,1,15,15
3,3,3,Excited about the upcoming weekend getaway! ...,Positive,2023-01-15 18:20:00,AdventureX,Facebook,#Travel #Adventure,8.0,15.0,UK,2023,1,15,18
4,4,4,Trying out a new recipe for dinner tonight. ...,Neutral,2023-01-15 19:55:00,ChefCook,Instagram,#Cooking #Food,12.0,25.0,Australia,2023,1,15,19


In [6]:
sentiment_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 732 entries, 0 to 731
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Unnamed: 0.1  732 non-null    int64  
 1   Unnamed: 0    732 non-null    int64  
 2   Text          732 non-null    object 
 3   Sentiment     732 non-null    object 
 4   Timestamp     732 non-null    object 
 5   User          732 non-null    object 
 6   Platform      732 non-null    object 
 7   Hashtags      732 non-null    object 
 8   Retweets      732 non-null    float64
 9   Likes         732 non-null    float64
 10  Country       732 non-null    object 
 11  Year          732 non-null    int64  
 12  Month         732 non-null    int64  
 13  Day           732 non-null    int64  
 14  Hour          732 non-null    int64  
dtypes: float64(2), int64(6), object(7)
memory usage: 85.9+ KB


In [7]:
sentiment_data = sentiment_data.drop(columns=['Unnamed: 0.1', 'Unnamed: 0', 'Timestamp', 'User', 'Platform', 'Hashtags', 'Retweets', 'Likes', 'Country', 'Year', 'Month', 'Day', 'Hour'])

### Working with Sentiment Labels

In [8]:
print("\nNo. of Unique Sentiments:", sentiment_data['Sentiment'].nunique())


No. of Unique Sentiments: 279


In [9]:
sentiment_data['Sentiment'].unique()

array([' Positive  ', ' Negative  ', ' Neutral   ', ' Anger        ',
       ' Fear         ', ' Sadness      ', ' Disgust      ',
       ' Happiness    ', ' Joy          ', ' Love         ',
       ' Amusement    ', ' Enjoyment    ', ' Admiration   ',
       ' Affection    ', ' Awe          ', ' Disappointed ',
       ' Surprise     ', ' Acceptance   ', ' Adoration    ',
       ' Anticipation ', ' Bitter       ', ' Calmness     ',
       ' Confusion    ', ' Excitement   ', ' Kind         ',
       ' Pride        ', ' Shame        ', ' Confusion ', ' Excitement ',
       ' Shame ', ' Elation       ', ' Euphoria      ', ' Contentment   ',
       ' Serenity      ', ' Gratitude     ', ' Hope          ',
       ' Empowerment   ', ' Compassion    ', ' Tenderness    ',
       ' Arousal       ', ' Enthusiasm    ', ' Fulfillment  ',
       ' Reverence     ', ' Compassion', ' Fulfillment   ', ' Reverence ',
       ' Elation   ', ' Despair         ', ' Grief           ',
       ' Loneliness     

In [10]:
# Stripping the leading and trailing spaces from each sentiment
sentiment_data['Sentiment'] = [sentiment.strip() for sentiment in sentiment_data['Sentiment']]

In [11]:
print("\nNo. of Unique Sentiments after stripping the unwanted spaces from sentiment labels :", sentiment_data['Sentiment'].nunique())


No. of Unique Sentiments after stripping the unwanted spaces from sentiment labels : 191


In [12]:
sentiment_counts = sentiment_data['Sentiment'].value_counts()
print("Sentiment Counts:\n")
sentiment_counts[:10]

Sentiment Counts:



,count
Sentiment,
Positive,45
Joy,44
Excitement,37
Contentment,19
Neutral,18
Gratitude,18
Curiosity,16
Serenity,15
Happy,14


### Checking for Duplicate Text Entries

In [13]:
print("No. of Duplicate Text Entries", sentiment_data['Text'].duplicated().sum())

No. of Duplicate Text Entries 25


In [14]:
# Filtering texts that appear more than once
duplicate_texts = sentiment_data.groupby('Text').size().reset_index(name='count')
duplicate_texts[duplicate_texts['count'] > 1]

,Text,count
1,"A compassionate rain, tears of empathy fallin...",3
4,"A free spirit soaring on the wings of dreams,...",2
12,"A playful escapade in the carnival of life, c...",3
40,"Bathed in the golden hues of gratefulness, a ...",2
52,Charting a course through the waves of hopefu...,2
59,"Confident strides in the dance of life, a bal...",2
81,"Dismissive gestures, a curtain drawn to shiel...",2
93,"Embraced by the hopeful dawn, a gardener sowi...",3
148,"Floating on clouds of inspiration, an artist ...",2
176,"Hopeful whispers of wind, carrying the promis...",2


### Handling Duplicate Text Entries

In [15]:
# Keeping the first occurrence of each duplicate
sentiment_data = sentiment_data.drop_duplicates(subset=['Text'], keep='first')

In [16]:
print("No. of Duplicate Text Entries (keeping only the first occurrence): ", sentiment_data['Text'].duplicated().sum())

No. of Duplicate Text Entries (keeping only the first occurrence):  0


In [17]:
sentiment_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 707 entries, 0 to 731
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Text       707 non-null    object
 1   Sentiment  707 non-null    object
dtypes: object(2)
memory usage: 16.6+ KB


## Data Preprocessing

### Encoding the sentiment labels

In [18]:
label_encoder = LabelEncoder()
sentiment_data['Encoded Sentiment'] = label_encoder.fit_transform(sentiment_data['Sentiment'])

### Applying Preprocessing to all the texts

In [20]:
nltk.download('stopwords')
nltk.download('punkt_tab')
nltk.download('wordnet')

def preprocess_text(text):
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    text = text.lower()
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word not in stopwords.words('english')]
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    return tokens

sentiment_data['Tokenized Text'] = [preprocess_text(text) for text in sentiment_data['Text'].values]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...


### Dataset after Preprocessing/Tokenization of texts and Label Encoding of Sentiments

In [21]:
sentiment_data.head(10)

,Text,Sentiment,Encoded Sentiment,Tokenized Text
0,Enjoying a beautiful day at the park! ...,Positive,146,"[enjoying, beautiful, day, park]"
1,Traffic was terrible this morning. ...,Negative,134,"[traffic, terrible, morning]"
2,Just finished an amazing workout! 💪 ...,Positive,146,"[finished, amazing, workout]"
3,Excited about the upcoming weekend getaway! ...,Positive,146,"[excited, upcoming, weekend, getaway]"
4,Trying out a new recipe for dinner tonight. ...,Neutral,135,"[trying, new, recipe, dinner, tonight]"
5,Feeling grateful for the little things in lif...,Positive,146,"[feeling, grateful, little, thing, life]"
6,Rainy days call for cozy blankets and hot coc...,Positive,146,"[rainy, day, call, cozy, blanket, hot, cocoa]"
7,The new movie release is a must-watch! ...,Positive,146,"[new, movie, release, mustwatch]"
8,Political discussions heating up on the timel...,Negative,134,"[political, discussion, heating, timeline]"
9,Missing summer vibes and beach days. ...,Neutral,135,"[missing, summer, vibe, beach, day]"


### Extracting the Relevant Columns

In [22]:
tokenized_texts = sentiment_data['Tokenized Text'].values
labels = sentiment_data['Encoded Sentiment'].values

### Word2Vec Training & Text Vectorization

#### Train custom Word2Vec model

In [25]:
word2vec_model = Word2Vec(
    sentences=tokenized_texts,  # List of tokenized sentences
    vector_size=100,            # Size of the word vectors
    window=5,                   # Maximum distance between the current and predicted word
    min_count=2,                # Ignores words with frequency lower than this
    workers=4,                  # Number of CPU cores
    sg = 1
)

In [26]:
word2vec_model.save("word2vec.model")

#### Word2Vec vocabulary details

In [27]:
print("Word2Vec vocabulary size:", len(word2vec_model.wv))
print("Sample words in vocabulary:", list(word2vec_model.wv.index_to_key)[:10])

Word2Vec vocabulary size: 978
Sample words in vocabulary: ['new', 'life', 'day', 'joy', 'moment', 'feeling', 'like', 'friend', 'heart', 'night']


In [32]:
# sample word vector
word2vec_model.wv['dream']

array([-0.00134503,  0.00749628,  0.01095018, -0.01083784, -0.00843607,
       -0.01750925, -0.00180087,  0.01882845, -0.00985678, -0.01170151,
        0.00149062, -0.01250158,  0.00785652, -0.00074926,  0.00553191,
       -0.00010326,  0.00367569, -0.01256671,  0.00342911, -0.02101866,
        0.00043042,  0.00021563,  0.01052013, -0.01023289, -0.00771293,
        0.00548167, -0.00563027, -0.01094416,  0.00374056,  0.00462585,
        0.00411291,  0.00455317,  0.01100338,  0.00785194,  0.00418933,
        0.01263985,  0.00027707, -0.00781501, -0.0131604 , -0.01429575,
        0.00093814, -0.00445574, -0.00603149,  0.00528949, -0.00315149,
       -0.0103952 , -0.00441027, -0.00483939, -0.00533885,  0.00661615,
       -0.00096278, -0.00697056, -0.0093878 ,  0.00331642, -0.00594855,
        0.00445199, -0.00104791, -0.00168783, -0.00209766,  0.00430703,
       -0.00645585, -0.00095533, -0.00142288, -0.00096768, -0.0190217 ,
        0.00868203, -0.00413591, -0.00284536, -0.00577498,  0.00

#### Converting the Tokenized texts into Sentence vectors

In [29]:
# using average of word vectors to represent sentences.
def text_to_vectors(text, model, vector_size=100):
    vectors = [model.wv[word] for word in text if word in model.wv]
    if len(vectors) > 0:
        return np.mean(vectors, axis=0)  # Average word vectors
    else:
        return np.zeros(vector_size)  # Return zero vector if no words are in vocabulary

In [30]:
vectorized_texts = np.array([text_to_vectors(text, model) for text in tokenized_texts])

In [31]:
vectorized_texts.shape

(707, 100)

In [33]:
print("First Sentence in Tokenized form:\n", tokenized_texts[0])
print("\nFirst Sentence in Vectorized form:\n", vectorized_texts[0])  # Printing the first 5 word vectors from the first sentence

First Sentence in Tokenized form:
 ['enjoying', 'beautiful', 'day', 'park']

First Sentence in Vectorized form:
 [-0.00513012  0.00556102 -0.00458171 -0.00411881 -0.00185008 -0.0036347
 -0.00493423  0.00625088 -0.00465942  0.00146757 -0.00988239 -0.00609408
  0.00271998  0.001852    0.00251819 -0.00564513  0.00096643  0.00044542
 -0.0010063  -0.00850558  0.00213788 -0.00097737 -0.00280217 -0.00342641
  0.00315668 -0.00055318 -0.00039253  0.00011234 -0.00105617  0.00550717
  0.00352085 -0.00223756  0.00176134 -0.00107218 -0.00067751  0.00136745
  0.00445134 -0.00372059 -0.00085777 -0.00333346 -0.00470237 -0.00290568
  0.00027674  0.00112983  0.00304642 -0.00339039  0.00042893 -0.00145852
 -0.00047056  0.00164184  0.0007676  -0.00551474 -0.00073693 -0.00200427
  0.00049124  0.00226406  0.00074432  0.00218045 -0.00711484 -0.00230741
 -0.0008423   0.00310927 -0.00239204  0.00160744 -0.00837686  0.00427378
  0.00692792  0.00294463 -0.00246845  0.00964445  0.00128749 -0.00122129
  0.00452331

### Preparing Data for Training

In [34]:
X = vectorized_texts
y = labels

In [35]:
class SentimentDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.long)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [36]:
# DataLoader for training
batch_size = 64
dataset = SentimentDataset(X, y)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)